In [6]:
# import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument("data_dir", help="What dir should I ue for generating data")
# parser.add_argument("--n_cores", type=int, help="Number of cores to use")
# args = parser.parse_args()
# DATA_DIR = args.data_dir
# N_CORES = 1
# if args.n_cores:
#     N_CORES = args.n_cores


DATA_DIR = "../../data"
N_CORES = 10

import geopandas as gpd
import numpy as np
import os
import pandas as pd
from utils.rasters import RasterGenerationHelper
from utils.shapefile import ShapefileHelper
import time
from pathlib import Path

TILE_DIR = f'{DATA_DIR}/interim/tiles'
PARENT_GDF = f'{DATA_DIR}/interim/parent.gpkg'
REDO_GDF = f'{DATA_DIR}/interim/redo.gpkg'

def get_missing():
    parent = gpd.read_file(PARENT_GDF)
    _all_ids = set(parent['pgrid_id'])
    _downloaded_ids = set([int(file.split(".tif")[0]) for file in os.listdir(TILE_DIR)])
    missing = all_ids - tiles
    missing_gdf = parent[parent['pgrid_id'].isin(list(missing))]
    missing_gdf.to_file(f"{DATA_DIR}/interim/redo.gpkg", driver="GPKG")
    return missing

complete = False
while not complete:
    missing = get_missing()
    if len(missing) > 0:
        print(f"#### Missing {len(missing)} tiles, redownloading..")
    
        _TILE_OUTPUT_DIR = f'{DATA_DIR}/interim/tiles'
        _PATH_TO_PARENT_GRID = f"{DATA_DIR}/interim/redo.gpkg"
        _PATH_TO_CHILD_GRID = f"{_VECTOR_OUTPUT_DIR}/child.gpkg"
        _N_CORES = N_CORES

    #     if os.path.exists(_TILE_OUTPUT_DIR):
    #         shutil.rmtree(_TILE_OUTPUT_DIR)

        Path(_TILE_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
        rgh = RasterGenerationHelper(_PATH_TO_PARENT_GRID, _PATH_TO_CHILD_GRID, _TILE_OUTPUT_DIR, _N_CORES, clean = False, post_period_days = [30,45])
        rgh.get_rasters()
    else:
        print(f"#### Missing 0 tiles, proceeding to merge..")
        complete = True


    

/home/arogya/.local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
*** Earth Engine *** FINAL DEADLINE: ee.Authenticate will fail after 2022-06-06. Please upgrade. https://developers.google.com/earth-engine/guides/python_install


In [9]:
# from utils.rasters import MergeRasterSingleAoi, Masker, Sampler
# from utils.data import ModelingHelper, DataHelper
from pathlib import Path
import os
import shutil

DATA_DIR = "../../data"
SHP_PATH = "/data/tmp/arogya/data/inputs/2306.gpkg"
MODEL_PATH = "/data/tmp/arogya/data/outputs/models/kmeans_3_diff_bands"
DATA_FILE_SUFFIX = "diff_bands"
TILES_PATH = f"{DATA_DIR}/interim/tiles"
YEAR = '2019'
NAME = MODEL_PATH.split("/")[-1]
SKIP_RASTER_GENERATION = False
N = 3
MODEL_TYPE = 'kmeans'
DIST = '2306'

# if args.year:
#     YEAR = args.year
# if args.name:
#     NAME = args.name
# if args.skip_raster_generation:
#     SKIP_RASTER_GENERATION = True
# if args.model_type:
#     MODEL_TYPE=args.model_type
# if args.num:
#     N=int(args.num)
# if args.dist:
#     DIST = args.dist
# if args.tiles_path:
#     TILES_PATH = arg.tiles_path

    


In [ ]:
if not SKIP_RASTER_GENERATION:
    
    if os.path.exists(f"{DATA_DIR}/interim/temp"):
        shutil.rmtree(f"{DATA_DIR}/interim/temp")

    if os.path.exists(f"{DATA_DIR}/interim/temp.vrt"):
        os.remove(f"{DATA_DIR}/interim/temp.vrt")
    
    if os.path.exists(f"{DATA_DIR}/interim/temp.tif"):
        os.remove(f"{DATA_DIR}/interim/temp.tif")
    
    if os.path.exists(f"{DATA_DIR}/interim/merged.tif"):
        os.remove(f"{DATA_DIR}/interim/merged.tif")
        
    if os.path.exists(f"{DATA_DIR}/interim/masked.tif"):
        os.remove(f"{DATA_DIR}/interim/masked.tif")
    
    
    print("In Merger")
    
    # Create mereged raster for AOI
    mrs = MergeRasterSingleAoi(DATA_DIR, SHP_PATH, TILES_PATH)
    mrs.merge("temp")

    # Create Masked Raster for AOI
    _CROP_MASK_PATH = f'{DATA_DIR}/inputs/{YEAR}_E060N40_PROBAV_LC100_global_v3.0.1_2019.tif'
    _INPUT_RASTER_PATH = f'{DATA_DIR}/interim/temp.tif'
    print("In Masker")

    masker = Masker(DATA_DIR, SHP_PATH, _INPUT_RASTER_PATH, _CROP_MASK_PATH)
    masker.mask("temp")

In Merger


In [ ]:

if not SKIP_RASTER_GENERATION:

    if os.path.exists(f"{DATA_DIR}/interim/temp.vrt"):
        os.remove(f"{DATA_DIR}/interim/temp.vrt")
    
    if os.path.exists(f"{DATA_DIR}/interim/temp.tif"):
        os.remove(f"{DATA_DIR}/interim/temp.tif")
    
    if os.path.exists(f"{DATA_DIR}/interim/merged.tif"):
        os.remove(f"{DATA_DIR}/interim/merged.tif")
        
    if os.path.exists(f"{DATA_DIR}/interim/masked.tif"):
        os.remove(f"{DATA_DIR}/interim/masked.tif")
    
    
    print("In Merger")
    
    # Create mereged raster for AOI
    mrs = MergeRasterSingleAoi(DATA_DIR, SHP_PATH, TILES_PATH)
    mrs.merge("temp")

    # Create Masked Raster for AOI
    _CROP_MASK_PATH = f'{DATA_DIR}/inputs/{YEAR}_E060N40_PROBAV_LC100_global_v3.0.1_2019.tif'
    _INPUT_RASTER_PATH = f'{DATA_DIR}/interim/temp.tif'
    print("In Masker")

    masker = Masker(DATA_DIR, SHP_PATH, _INPUT_RASTER_PATH, _CROP_MASK_PATH)
    masker.mask("temp")

print("In Sampler")
sampler = Sampler(DATA_DIR, f'{DATA_DIR}/interim/temp.tif')
sampler.sample(1, sample_filename = "temp", save_full=False)

dh = DataHelper(DATA_DIR, f'{DATA_DIR}/interim/temp.tgz')

if DATA_FILE_SUFFIX=="diff_bands":
    dh.save(dh.diff_bands(), "diff_bands", filename=f'{DATA_DIR}/interim/temp.tgz')
if DATA_FILE_SUFFIX=="pre_only":
    dh.save(dh.pre_only(), "pre_only", filename=f'{DATA_DIR}/interim/temp.tgz')
if DATA_FILE_SUFFIX=="post_only":
    dh.save(dh.post_only(), "post_only", filename=f'{DATA_DIR}/interim/temp.tgz')
if DATA_FILE_SUFFIX=="ndvi_and_day":
    dh.save(dh.ndvi_and_day(), "ndvi_and_day", filename=f'{DATA_DIR}/interim/temp.tgz')
if DATA_FILE_SUFFIX=="diff_all":
    dh.save(dh.ndvi_and_day(), "diff_all", filename=f'{DATA_DIR}/interim/temp.tgz')

    
    
_DATA_FILE_PATH = f'{DATA_DIR}/interim/temp.tgz'
mh = ModelingHelper(DATA_DIR, _DATA_FILE_PATH, "dummy")
model = mh.load_model(f"{MODEL_PATH}/model.pickle")
scaler = mh.load_scaler(f"{MODEL_PATH}/scaler.pickle")

results = mh.predict(mh.raw, model, scaler)

Path(f"{DATA_DIR}/outputs/predictions/{NAME}").mkdir(parents=True, exist_ok=True)

mh.save_raster(results, f"{DATA_DIR}/outputs/predictions/{NAME}/{NAME}")
mh.save_comparison_results(results, DIST, YEAR, f"{DATA_DIR}/outputs/predictions/{NAME}/{NAME}")
mh.save_ndvi_plot(results, MODEL_TYPE, N, f"{DATA_DIR}/outputs/predictions/{NAME}/{NAME}")

mrs=None
masker=None
sampler=None
dh=None
mh=None
model=None
scaler=None
results=None
gc.collect(generation=2)